In [18]:
import pandas as pd
import boto3
from StringIO import StringIO
pd.options.mode.chained_assignment = None

### Init s3 client

In [4]:
s3 = boto3.client('s3') 

### Load Tickets

In [5]:
tickets_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/tickets.csv')['Body'].read().decode('utf-8')
tickets = pd.read_csv(StringIO(tickets_csv_string), header=0, delimiter='|')
tickets['meal_created_date'] = pd.to_datetime(tickets['meal_created_date'])
tickets['meal_date'] = pd.to_datetime(tickets['meal_date'])

### Load Meals

In [7]:
meals_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meals.csv')['Body'].read().decode('utf-8')
meals = pd.read_csv(StringIO(meals_csv_string), header=0)

### Load Cooks

In [8]:
cooks_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/cooks.csv')['Body'].read().decode('utf-8')
cooks = pd.read_csv(StringIO(cooks_csv_string), header=0)

### Load Menus

In [9]:
menus_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/menus.csv')['Body'].read().decode('utf-8')
menus = pd.read_csv(StringIO(menus_csv_string), header=0)

### Load Venues

In [10]:
venues_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/venues.csv')['Body'].read().decode('utf-8')
venues = pd.read_csv(StringIO(venues_csv_string), header=0)

### Load Menu Course Counts

In [11]:
menu_course_counts_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/menu_course_counts.csv')['Body'].read().decode('utf-8')
menu_course_counts = pd.read_csv(StringIO(menu_course_counts_csv_string), header=0, delimiter='|')

### Load Meal Addon Counts

In [12]:
meal_addon_counts_csv_string = s3.get_object(Bucket='braydencleary-data', Key='feastly/cleaned/meal_addon_counts.csv')['Body'].read().decode('utf-8')
meal_addon_counts = pd.read_csv(StringIO(meal_addon_counts_csv_string), header=0, delimiter='|')

### Filter meals to only include those present in tickets df

In [13]:
meals = meals[meals.id.isin(tickets['meal_id'].unique())]

### Set initial feature matrix (to be expanded on as notebook progresses)

##### Including meal_id, percentage_of_seats_sold, and sold (target) in feature matrix now but will remove later

In [14]:
X = tickets[['meal_id', 'percentage_of_seats_sold', 'sold', 'number_of_seats', 'days_to_sell', 'ticket_price']]

### Calculate listed_days feature and add to feature matrix

In [19]:
tickets['listed_days'] = (tickets['meal_date'] - tickets['meal_created_date'])
tickets['listed_days'] = tickets['listed_days'].apply(lambda listed_days: listed_days.days)
X['meal_listed_days'] = tickets['listed_days']